In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse 
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# データセット読み込み
(x_train, y_train), (x_test, y_test) = mnist.load_data()
image_size = x_train.shape[1] # = 784
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

input_shape = (original_dim, )
latent_dim = 2

# Reparametrization Trick 
def sampling(args):
    z_mean, z_logvar = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim), seed = 5) # ε
    return z_mean + K.exp(0.5 * z_logvar) * epsilon

# VAEモデル構築
inputs = Input(shape=input_shape)
x1 = Dense(256, activation='relu')(inputs)  
x2 = Dense(64, activation='relu')(x1) 
z_mean = Dense(latent_dim)(x2)
z_logvar = Dense(latent_dim)(x2)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_logvar])
encoder = Model(inputs, [z_mean, z_logvar, z], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,))
x3 = Dense(64, activation='relu')(latent_inputs)  
x4 = Dense(256, activation='relu')(x3)  
outputs = Dense(original_dim, activation='sigmoid')(x4)
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

z_output = encoder(inputs)[2]
outputs = decoder(z_output)
vae = Model(inputs, outputs, name='variational_autoencoder')

# 損失関数
# Kullback-Leibler Loss
kl_loss = 1 + z_logvar - K.square(z_mean) - K.exp(z_logvar)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
# Reconstruction Loss
reconstruction_loss = mse(inputs, outputs)
reconstruction_loss *= original_dim

vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          200960      input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           16448       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2)            130         dense_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
vae.compile(optimizer='adam')
vae.fit(x_train,
                epochs=50,
                batch_size=256,
                validation_data=(x_test, None))

Epoch 1/50
235/235 [==============================] - 4s 17ms/step - loss: 57.5695 - val_loss: 45.9800
Epoch 2/50
235/235 [==============================] - 4s 17ms/step - loss: 44.7460 - val_loss: 43.5027
Epoch 3/50
235/235 [==============================] - 4s 19ms/step - loss: 43.0082 - val_loss: 42.2514
Epoch 4/50
235/235 [==============================] - 4s 17ms/step - loss: 42.0338 - val_loss: 41.4285
Epoch 5/50
235/235 [==============================] - 4s 16ms/step - loss: 41.2499 - val_loss: 40.7275
Epoch 6/50
235/235 [==============================] - 4s 16ms/step - loss: 40.6262 - val_loss: 40.1753
Epoch 7/50
235/235 [==============================] - 4s 16ms/step - loss: 40.0916 - val_loss: 39.6940
Epoch 8/50
235/235 [==============================] - 4s 17ms/step - loss: 39.6249 - val_loss: 39.2167
Epoch 9/50
235/235 [==============================] - 4s 16ms/step - loss: 39.2122 - val_loss: 38.8890
Epoch 10/50
235/235 [==============================] - 4s 17ms/step - los